# import

In [1]:
import csv
import os
import sys
from concurrent.futures import ProcessPoolExecutor
from glob import glob, iglob

import numpy as np
import pandas as pd
import xmltodict
from tqdm import tqdm
from functools import reduce

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
tqdm.pandas()

# Read data

In [4]:
path = glob("../../data/NewYorkTimes/parsed/*")[0]

In [5]:
path

'../../data/NewYorkTimes/parsed/1987.csv'

In [6]:
df = pd.read_csv(path, index_col=0)

In [7]:
df.isna().sum()

nitf.head.meta                           0
nitf.head.pubdata.@date.publication      0
full_text                                0
classifier                             342
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.classifier = df.classifier.progress_apply(lambda x: x.split('%'))

100%|██████████| 105760/105760 [00:00<00:00, 111909.62it/s]


In [10]:
classes = df.classifier.progress_apply(lambda x: set(x))

100%|██████████| 105760/105760 [00:01<00:00, 103914.74it/s]


In [11]:
classes = classes.tolist()

In [12]:
classes = reduce(lambda total, x: total | x,classes, set())

In [13]:
classes_sep = [c.split("/") for c in classes]

In [32]:
layer_num = 3

In [33]:
layerd_labels = list(filter(lambda x: len(x)==layer_num, classes_sep))

In [34]:
layerd_labels = ["/".join(label) for label in layerd_labels]

In [36]:
df[f"classifier_{layer_num}"] = df["classifier"].progress_apply(
    lambda x: set(x) & set(layerd_labels)
)

100%|██████████| 105760/105760 [00:01<00:00, 72749.15it/s]


In [47]:
len(layerd_labels)

25

In [46]:
df

,nitf.head.meta,nitf.head.pubdata.@date.publication,full_text,classifier,classifier_3
0,"[{'@content': '1', '@name': 'publication_day_o...",19870101T000000,LEAD: *3*** COMPANY REPORTS ** *3*AAR CORP (NY...,[Top/News/Business],{Top/News/Business}
0,"[{'@content': '1', '@name': 'publication_day_o...",19870101T000000,LEAD: *3*** COMPANY REPORTS ** *3*AMERICAN CYT...,[Top/News/Business],{Top/News/Business}
0,"[{'@content': '1', '@name': 'publication_day_o...",19870101T000000,LEAD: *3*** COMPANY REPORTS ** *3*APPLIED POWE...,[Top/News/Business],{Top/News/Business}
0,"[{'@content': '1', '@name': 'publication_day_o...",19870101T000000,LEAD: *3*** COMPANY REPORTS ** *3*BARRETT RESO...,[Top/News/Business],{Top/News/Business}
0,"[{'@content': '1', '@name': 'publication_day_o...",19870101T000000,LEAD: *3*** COMPANY REPORTS ** *3*CANAVERAL IN...,[Top/News/Business],{Top/News/Business}
...,...,...,...,...,...
0,"[{'@content': '31', '@name': 'publication_day_...",19871231T000000,LEAD: John McEnroe said yesterday that he woul...,"[Top/News, Top/News/Sports, Top/News/Sports/Te...",{Top/News/Sports}
0,"[{'@content': '31', '@name': 'publication_day_...",19871231T000000,LEAD: A Federal judge in Minneapolis yesterday...,"[Top/News, Top/News/Sports, Top/News/Sports/Pr...",{Top/News/Sports}
0,"[{'@content': '31', '@name': 'publication_day_...",19871231T000000,LEAD: Damage hearings in the 1985 baseball col...,"[Top/News, Top/News/Sports, Top/News/Sports/Ba...",{Top/News/Sports}
0,"[{'@content': '31', '@name': 'publication_day_...",19871231T000000,"LEAD: Don Nelson, now an executive with the Go...","[Top/News, Top/News/Sports, Top/News/Sports/Pr...",{Top/News/Sports}
